# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# get abs path of each csv file
print(os.getcwd())
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/bing/data-engineer/NoSQL-Sparkify


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# read csv file
full_data_rows_list = [] 
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader) # skil header
         
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# print(len(full_data_rows_list))
# print(full_data_rows_list)

# define dialect and merge csv files into event_datafile_new.csv
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS mykeyspace
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('mykeyspace')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Task: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
use `(sessionId, itemInSession)` as primary key, where sessionId as partition key

In [8]:
query = """
CREATE TABLE IF NOT EXISTS music_library
(sessionId INT, itemInSession INT, artist TEXT, song_title TEXT, song_length float, 
PRIMARY KEY(sessionId, itemInSession))
"""

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
# insert data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_library (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[-3]), int(line[3]), line[0], line[-2], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# verify the data 
query = """
SELECT artist, song_title, song_length FROM music_library 
WHERE sessionid = 338 AND iteminsession = 4;
"""

try:
    rows = session.execute(query)
    df = pd.DataFrame(list(rows))
    print(df)
except Exception as e:
    print(e)

      artist                       song_title  song_length
0  Faithless  Music Matters (Mark Knight Dub)   495.307312


#### Task: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

- `((userId, sessionId), itemInSession)` as primary key to keep song sorted by itemInSession

- `(userId, sessionId)` as partition key to allow filtering by sessionId 

In [11]:
query = """
CREATE TABLE IF NOT EXISTS user_library
(userid INT, sessionId INT, itemInSession INT, artist TEXT, song_title TEXT, user TEXT,
PRIMARY KEY ((userid, sessionId), itemInSession)
)
"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# insert data
query = """
INSERT INTO user_library (userid, sessionId, itemInSession, artist, song_title, user)
VALUES (%s, %s, %s, %s, %s, %s)
"""

with open(file, encoding='utf8') as f:
    cr = csv.reader(f)
    next(cr)
    for line in cr:
        try:
            session.execute(query, (int(line[-1]), int(line[-3]), int(line[3]), line[0], line[-2], line[1]+' '+line[4]))
        except Exception as e:
            print(e)


In [13]:
# validate data
query = """
SELECT artist, song_title, user from user_library 
WHERE userid = 10 AND sessionid = 182;
"""

try:
    rows = session.execute(query)
    df = pd.DataFrame(list(rows))
    print(df)
except Exception as e:
    print(e)


              artist                                         song_title  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

          user  
0  Sylvie Cruz  
1  Sylvie Cruz  
2  Sylvie Cruz  
3  Sylvie Cruz  


#### Task: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
`(song, userId)` uniquely identify the query

In [14]:
query = """
CREATE TABLE IF NOT EXISTS user_song
(song TEXT, userId INT, user TEXT, PRIMARY KEY (song, userId))
"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
query = """
INSERT INTO user_song (user, userId, song)
VALUES (%s, %s, %s)
"""

with open(file, encoding='utf8') as f:
    cr = csv.reader(f)
    next(cr) # skip header
    for line in cr:
        try:
            session.execute(query, (line[1]+' '+line[4], int(line[-1]), line[-2]))
        except Exception as e:
            print(e)


In [16]:
query = """
SELECT user from user_song 
WHERE song='All Hands Against His Own';
"""

try:
    rows = session.execute(query)
    df = pd.DataFrame(list(rows))
    print(df)
except Exception as e:
    print(e)


               user
0  Jacqueline Lynch
1      Tegan Levine
2      Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
# Drop the table before closing out the sessions
drop0 = "DROP TABLE IF EXISTS user_library"
drop1 = "DROP TABLE IF EXISTS music_library"
drop2 = "DROP TABLE IF EXISTS user_song"

drops = [drop0, drop1, drop2]

for q in drops:
    try:
        session.execute(q)
    except Exception as e:
        print(e)


### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()